In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nbme-score-clinical-patient-notes/sample_submission.csv
/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv
/kaggle/input/nbme-score-clinical-patient-notes/features.csv
/kaggle/input/nbme-score-clinical-patient-notes/train.csv
/kaggle/input/nbme-score-clinical-patient-notes/test.csv
/kaggle/input/nbme-preprocessed-sentences/patient_notes_sentences.csv
/kaggle/input/nbme-preprocessed-sentences/__notebook_source__.ipynb
/kaggle/input/nbme-preprocessed-sentences/data_train_sentence.csv


# MLM Pre-training
Credit to : [https://github.com/UKPLab/sentence-transformers/blob/master/examples/unsupervised_learning/MLM/train_mlm.py](http://)

In [2]:
from transformers import AutoModelForMaskedLM, AutoTokenizer
from transformers import DataCollatorForLanguageModeling, DataCollatorForWholeWordMask
from transformers import Trainer, TrainingArguments
import sys
import gzip
from datetime import datetime

model_name = 'emilyalsentzer/Bio_ClinicalBERT'
per_device_train_batch_size = 64

os.environ["WANDB_DISABLED"] = "true"
save_steps = 1000               #Save model every 1k steps
num_train_epochs = 7            #Number of epochs
use_fp16 = True                #Set to True, if your GPU supports FP16 operations
max_length = 100                #Max length for a text input
do_whole_word_mask = True       #If set to true, whole words are masked
mlm_prob = 0.15                 #Probability that a word is replaced by a [MASK] token

# Load the model
model = AutoModelForMaskedLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

output_dir = "MLMPredtrained/{}-{}".format(model_name.replace("/", "_"),  datetime.now().strftime("%Y-%m-%d_%H-%M-%S"))
print("Save checkpoints to:", output_dir)

##### Load our training datasets
train_path = '/kaggle/input/nbme-preprocessed-sentences/patient_notes_sentences.csv'
train_sentences = pd.read_csv(train_path).sentence.tolist()
print("Train sentences:", len(train_sentences))

dev_sentences = []
print("Dev sentences:", len(dev_sentences))

#A dataset wrapper, that tokenizes our data on-the-fly
class TokenizedSentencesDataset:
    def __init__(self, sentences, tokenizer, max_length, cache_tokenization=False):
        self.tokenizer = tokenizer
        self.sentences = sentences
        self.max_length = max_length
        self.cache_tokenization = cache_tokenization

    def __getitem__(self, item):
        if not self.cache_tokenization:
            return self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)

        if isinstance(self.sentences[item], str):
            self.sentences[item] = self.tokenizer(self.sentences[item], add_special_tokens=True, truncation=True, max_length=self.max_length, return_special_tokens_mask=True)
        return self.sentences[item]

    def __len__(self):
        return len(self.sentences)

train_dataset = TokenizedSentencesDataset(train_sentences, tokenizer, max_length)
dev_dataset = TokenizedSentencesDataset(dev_sentences, tokenizer, max_length, cache_tokenization=True) if len(dev_sentences) > 0 else None


##### Training arguments
if do_whole_word_mask:
    data_collator = DataCollatorForWholeWordMask(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)
else:
    data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=True, mlm_probability=mlm_prob)

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_train_epochs,
    evaluation_strategy="steps" if dev_dataset is not None else "no",
    per_device_train_batch_size=per_device_train_batch_size,
    eval_steps=save_steps,
    save_steps=save_steps,
    logging_steps=save_steps,
    save_total_limit=1,
    prediction_loss_only=True,
    fp16=use_fp16
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    eval_dataset=dev_dataset
)

print("Save tokenizer to:", output_dir)
tokenizer.save_pretrained(output_dir)

trainer.train()

print("Save model to:", output_dir)
model.save_pretrained(output_dir)

print("Training done")

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Save checkpoints to: MLMPredtrained/emilyalsentzer_Bio_ClinicalBERT-2022-04-10_06-44-23


...

Step,Training Loss
1000,1.565200
2000,1.266400
3000,1.178200
4000,1.120200
5000,1.084700
6000,1.054900
7000,1.037000
8000,1.023500
9000,1.000500
10000,0.979100


Saving model checkpoint to MLMPredtrained/emilyalsentzer_Bio_ClinicalBERT-2022-04-10_06-44-23/checkpoint-1000
Configuration saved in MLMPredtrained/emilyalsentzer_Bio_ClinicalBERT-2022-04-10_06-44-23/checkpoint-1000/config.json
Model weights saved in MLMPredtrained/emilyalsentzer_Bio_ClinicalBERT-2022-04-10_06-44-23/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to MLMPredtrained/emilyalsentzer_Bio_ClinicalBERT-2022-04-10_06-44-23/checkpoint-2000
Configuration saved in MLMPredtrained/emilyalsentzer_Bio_ClinicalBERT-2022-04-10_06-44-23/checkpoint-2000/config.json
Model weights saved in MLMPredtrained/emilyalsentzer_Bio_ClinicalBERT-2022-04-10_06-44-23/checkpoint-2000/pytorch_model.bin
Deleting older checkpoint [MLMPredtrained/emilyalsentzer_Bio_ClinicalBERT-2022-04-10_06-44-23/checkpoint-1000] due to args.save_total_limit
Saving model checkpoint to MLMPredtrained/emilyalsentzer_Bio_ClinicalBERT-2022-04-10_06-44-23/checkpoint-3000
Configuration saved in MLMPredtrained/emilyals

Save model to: MLMPredtrained/emilyalsentzer_Bio_ClinicalBERT-2022-04-10_06-44-23


Model weights saved in MLMPredtrained/emilyalsentzer_Bio_ClinicalBERT-2022-04-10_06-44-23/pytorch_model.bin


Training done
